In [95]:
# Importing libraries
import pandas as pd
import numpy as np
import requests
from time import sleep
from random import randint
from bs4 import BeautifulSoup
from requests import get

# Creating variables for HTML tags and classes
tags = ['span', 'span', 'div','span','a']
classes = ['property-card__title js-cardLink js-card-title', 
           'property-card__address', 
           'property-card__price js-property-card-prices js-property-card__price-small', 
           'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area', 
           'property-card__content-link js-card-title'] 
                   
# Empty lists to store the scraped data
a_size = len(classes)
lists = [[] for i in range(a_size)]
link = []
 
# For loop to get the index of the website's last page 
last_page = []

page_index = pd.Series(soup.find_all('a', class_='js-change-page'))
for page in page_index:
    page.get('data-page')
    last_page.append(page.get_text())
    
# For loop to go through multiple pages and retrieve the data
pages = np.arange(1,pd.to_numeric(last_page[-2]),1)

for page in pages:
    page = requests.get("https://www.vivareal.com.br/aluguel/?pagina="+str(page))
    soup = BeautifulSoup(page.text, 'html.parser')    
    house_data = soup.find_all('article', class_="property-card__container js-property-card")
    for i in range(a_size):
        for data in soup.find_all(tags[i], class_=classes[i]):
            if tags[i] == 'a':
                lists[i].append(data['href'])
            else:
                lists[i].append(data.get_text())
                                            
## Creating Data Frames to treat the data ##      
df = pd.DataFrame({'descrição':lists[0],
                   'localização':lists[1],
                   'preço':lists[2],
                   'atc':lists[3],
                   'link':lists[4]})
# Type
df['tipo_imovel'] = df['descrição'].astype(str).str.split().str[0]
df['tipo_compra'] = 'Aluguel'
df = df.drop(['descrição'], axis=1)
# Address
df['endereço'] = df['localização'].str.split('-').str[0] 
df['cidade'] = df['localização'].str.split('-').str[1].str.split(',').str[1]
df['bairro'] = df['localização'].str.split('-').str[1].str.split(',').str[0]
df['uf'] = df['localização'].str.split('-').str[2] 
df = df.drop(['localização'], axis=1)
# Link
df['dominio'] = 'https://www.vivareal.com.br'
df['link'] = df['dominio'] + df['link']
df = df.drop(['dominio'], axis=1)

df.to_excel('relatorio_vivareal.xlsx')